In [7]:
import torch
import evaluate
import os
import csv
from tqdm import tqdm

def load_hypos(file_path, source_file_path):
    """
    Load texts from a file and return as a list of stripped strings.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        #also read  source file to get the source text
        with open(source_file_path, 'r', encoding='utf-8') as f_source:
            line = [line.strip() for line in f.readlines()]
            line_source = [line.strip() for line in f_source.readlines()]
            #the source line contains a # , save the content before and after the # in two different vars
            line_source = [line.split('#') for line in line_source]
            #remove both elements of the list line_source from the line of the hypothesis
            for i in range(len(line_source)):
                line[i] = line[i].replace(line_source[i][0], '')
                line[i] = line[i].replace(line_source[i][2], '')

        return line
    
def load_targets(file_path, source_file_path):
    """
    Load texts from a file and return as a list of stripped strings.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        #also read  source file to get the source text
        with open(source_file_path, 'r', encoding='utf-8') as f_source:
            line = [line.strip() for line in f.readlines()]
            line_source = [line.strip() for line in f_source.readlines()]
            #the source line contains a # , save the content before and after the # in two different vars
            line_source = [line.split('#') for line in line_source]
            #remove both elements of the list line_source from the line of the hypothesis
            for i in range(len(line_source)):
                line[i] = line[i].replace(line_source[i][0], '')
                line[i] = line[i].replace(line_source[i][2], '')

        return line

def main():
    # Define explicit pairs of hypothesis and target files
    # Format: (hypothesis_file_path, target_file_path)
    file_pairs = [
        ("generated_hypotheses/ikatdataparacomet_checkpoint16.hypo", "D3test/ikatdata.target", "D3test/ikatdataparacomet.source"),
        ("generated_hypotheses/jandataparacomet_checkpoint16.hypo", "D2test/jandata.target", "D2test/jandataparacomet.source"),
        ("generated_hypotheses/semevaldataparacomet_checkpoint16.hypo", "D1test/semevaldata.target", "D1test/semevaldataparacomet.source"),
    ]
    
    # Output CSV file to store evaluation results
    output_csv = 'evaluation_results.csv'
    
    # Initialize metrics
    bleu = evaluate.load('bleu')
    bertscore = evaluate.load('bertscore')
    rouge = evaluate.load('rouge')  # Optional, if ROUGE is desired
    
    # Prepare CSV header
    csv_header = ['Model', 'Source File', 'BLEU-1', 'BLEU-2', 
                  'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 
                  'BERTScore Precision', 'BERTScore Recall', 'BERTScore F1']
    
    # Open CSV file for writing
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(csv_header)
        
        print(f"Starting evaluation of {len(file_pairs)} file pairs.\n")
        
        # Iterate over each file pair with a progress bar
        for hypo_path, target_path, source_path in tqdm(file_pairs, desc="Evaluating file pairs"):
            # Extract model identifier and source file name from the hypothesis file name
            hypo_filename = os.path.basename(hypo_path)
            try:
                sourcefile, modelid = hypo_filename.split('_checkpoint')
                modelid = f"checkpoint{modelid.split('.hypo')[0]}"
            except Exception as e:
                print(f"Filename format error for {hypo_filename}: {e}")
                continue  # Skip this pair if filename is not as expected
            
            # Check if both files exist
            if not os.path.exists(hypo_path):
                print(f"Hypothesis file not found: {hypo_path}. Skipping.")
                continue
            if not os.path.exists(target_path):
                print(f"Target file not found: {target_path}. Skipping.")
                continue
            
            # Load texts
            generated_hypo_texts = load_hypos(hypo_path, source_path)
            target_texts = load_targets(target_path, source_path)
            # print 5 first elements of the target and hypothesis texts
            print(f"Target texts: {target_texts[:1]}")
            print(f"Hypothesis texts: {generated_hypo_texts[:1]}")  

            # Validate that all datasets have the same number of samples
            if len(target_texts) != len(generated_hypo_texts):
                print(f"Line count mismatch between hypothesis and target for {hypo_filename}. Skipping.")
                continue  # Skip if line counts do not match
            
            # Compute BLEU-1
            bleu1 = bleu.compute(
                predictions=generated_hypo_texts,
                references=[[ref] for ref in target_texts],
                max_order=1
            )
            
            # Compute BLEU-2
            bleu2 = bleu.compute(
                predictions=generated_hypo_texts,
                references=[[ref] for ref in target_texts],
                max_order=2
            )
            
            # Compute ROUGE
            ro = rouge.compute(
                predictions=generated_hypo_texts,
                references=target_texts,
                use_stemmer=True
            )
            
            # Compute BERTScore
            bs = bertscore.compute(
                predictions=generated_hypo_texts,
                references=target_texts,
                lang='en',
                model_type='bert-base-uncased',
                verbose=False
            )
            
            # Calculate average BERTScore metrics
            bert_precision = sum(bs['precision']) / len(bs['precision']) * 100
            bert_recall = sum(bs['recall']) / len(bs['recall']) * 100
            bert_f1 = sum(bs['f1']) / len(bs['f1']) * 100
            
            # Write results to CSV
            writer.writerow([
                modelid,
                sourcefile,
                f"{bleu1['bleu'] * 100:.2f}",
                f"{bleu2['bleu'] * 100:.2f}",
                f"{ro['rouge1'] * 100:.2f}",
                f"{ro['rouge2'] * 100:.2f}",
                f"{ro['rougeL'] * 100:.2f}",
                f"{bert_precision:.2f}",
                f"{bert_recall:.2f}",
                f"{bert_f1:.2f}"
            ])
            
            # Print summary for this pair
            print(f"Evaluated {hypo_filename}: BLEU-1={bleu1['bleu'] * 100:.2f}, BLEU-2={bleu2['bleu'] * 100:.2f}, ROUGE-1={ro['rouge1'] * 100:.2f}, ROUGE-2={ro['rouge2'] * 100:.2f}, ROUGE-L={ro['rougeL'] * 100:.2f}, BERTScore F1={bert_f1:.2f}")
        
        print(f"\nEvaluation completed. Results saved to {output_csv}.")

# Execute the evaluation
main()


Starting evaluation of 3 file pairs.



Evaluating file pairs:   0%|          | 0/3 [00:00<?, ?it/s]

Target texts: ['And since Computer games do not originate from old traditions.']
Hypothesis texts: ['And since She decided to go to the doctor. computer should also not be recognized as Olympic events.']


Evaluating file pairs:  33%|███▎      | 1/3 [00:01<00:02,  1.46s/it]

Evaluated ikatdataparacomet_checkpoint16.hypo: BLEU-1=17.04, BLEU-2=9.64, ROUGE-1=26.13, ROUGE-2=9.75, ROUGE-L=20.94, BERTScore F1=52.27
Target texts: ['[\'"He" stands for Obama\', \'Getting troops out of countries impies that wars are ending.\', \'"He" stands for Obama.\', \'Wars are conducted with troops\', \'The war refers to Iraq.\', "With no troops there\'s no war", \'Obama stated that he ended the war in Iraq.\', \'Obama pulled all US troops out of Iraq.\']']
Hypothesis texts: ['And since He decided to go to the store and had a war in the house. wars']


Evaluating file pairs:  67%|██████▋   | 2/3 [00:02<00:01,  1.35s/it]

Evaluated jandataparacomet_checkpoint16.hypo: BLEU-1=7.67, BLEU-2=3.50, ROUGE-1=23.75, ROUGE-2=5.97, ROUGE-L=16.64, BERTScore F1=43.98
Target texts: ["And since that helps the company's bottom line."]
Hypothesis texts: ['And since He decided to go to the store and get a lot of money to get a lot of money to get a new job. Unpaid exploit college students students']


Evaluating file pairs: 100%|██████████| 3/3 [00:07<00:00,  2.42s/it]

Evaluated semevaldataparacomet_checkpoint16.hypo: BLEU-1=16.33, BLEU-2=8.58, ROUGE-1=22.45, ROUGE-2=8.12, ROUGE-L=18.90, BERTScore F1=49.12

Evaluation completed. Results saved to evaluation_results.csv.
